<a href="https://colab.research.google.com/github/sylee20/kaggle_competition/blob/main/my_american_express_default_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dependencies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import os, gc

# Predefine functions

In [ ]:
# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
      col_type = df[col].dtype

      if col_type != object:
        c_min = df[col].min()
        c_max = df[col].max()
        if str(col_type)[:3] == 'int':
          if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
            df[col] = df[col].astype(np.int8)
          elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
            df[col] = df[col].astype(np.int16)
          elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
            df[col] = df[col].astype(np.int32)
          elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
            df[col] = df[col].astype(np.int64)
        else:
          if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
            df[col] = df[col].astype(np.float32)
          else: 
            df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB --> {:.2f} MB (Decreased by {:.1f}%)'.format(
        start_mem, end_mem, 100*(start_mem - end_mem) / start_mem))
    return df

#Load data

In [ ]:
 ! pip install -q kaggle

In [ ]:
from google.colab import files
my_upload = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                             title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
victorsoeiro/netflix-tv-shows-and-movies                        Netflix TV Shows and Movies                           2MB  2022-05-15 00:01:23           9378        284  1.0              
devansodariya/student-performance-data                          Student Performance Dataset                           7KB  2022-05-26 13:55:09           4977        160  0.9705882        
mohamedharris/supermart-grocery-sales-retail-analytics-dataset  Supermart Grocery Sales - Retail Analytics Dataset  191KB  2022-06-12 16:14:44            678         42  0.88235295       
sameepvani/nasa-nearest-earth-objects                       

In [ ]:
! kaggle datasets download -d raddar/amex-data-integer-dtypes-parquet-format

100% 4.07G/4.07G [00:23<00:00, 132MB/s]
100% 4.07G/4.07G [00:23<00:00, 184MB/s]


In [ ]:
! mkdir input

In [ ]:
! unzip amex-data-integer-dtypes-parquet-format.zip -d input

Archive:  amex-data-integer-dtypes-parquet-format.zip
  inflating: input/test.parquet      
  inflating: input/train.parquet     


In [ ]:
print(os.listdir('./input'))

['train.parquet', 'test.parquet']


In [ ]:
my_upload = files.upload()

Saving train_labels.csv to train_labels.csv


In [ ]:
! mv train_labels.csv input

In [ ]:
train_df = pd.read_parquet('./input/train.parquet')
test_df = pd.read_parquet('./input/test.parquet')
train_labels_df = pd.read_csv('./input/train_labels.csv')
print('train_df shape: ', train_df.shape)
print('train_labels_df shape: ', train_labels_df.shape)
print('test_df shape: ', test_df.shape)

train_df shape:  (5531451, 190)
train_labels_df shape:  (458913, 2)
test_df shape:  (11363762, 190)


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531451 entries, 0 to 5531450
Columns: 190 entries, customer_ID to D_145
dtypes: float32(93), int16(9), int8(86), object(2)
memory usage: 2.5+ GB


In [ ]:
train_df.describe()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
count,5.485466e+06,5.531451e+06,5.531451e+06,5.529435e+06,5.531451e+06,4.510907e+06,5.529435e+06,5.529435e+06,791314.000000,3.873055e+06,...,5.531451e+06,5.531451e+06,5.531451e+06,5.531451e+06,5.531451e+06,5.429903e+06,944408.000000,5.531451e+06,5.490724e+06,5.531451e+06
mean,6.563343e-01,5.035986e+00,1.240101e-01,6.214890e-01,7.880273e-02,2.258454e-01,5.543458e-02,1.325390e-01,0.184974,1.546841e-01,...,-9.313243e-01,-9.644764e-01,-9.535646e-01,1.523781e-01,1.413933e-02,1.603911e-01,0.390799,1.522530e-01,5.238953e-02,6.007445e-01
std,2.446494e-01,9.181833e+00,2.119869e-01,4.014876e-01,2.263971e-01,1.933475e-01,2.037066e-01,2.349929e-01,0.228185,2.133977e-01,...,3.926067e-01,1.868469e-01,2.629447e-01,4.072783e-01,1.692060e-01,3.501590e-01,0.236182,4.071715e-01,1.825135e-01,2.119894e+00
min,-4.589548e-01,0.000000e+00,-7.588799e+00,9.192280e-09,1.534223e-09,-6.271321e-01,0.000000e+00,6.285293e-09,-0.000454,1.154550e-07,...,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,-0.014539,-1.000000e+00,2.500991e-09,-1.000000e+00
25%,4.803307e-01,0.000000e+00,8.863645e-03,1.053313e-01,2.895934e-03,1.272588e-01,0.000000e+00,5.227570e-03,0.037516,4.227546e-02,...,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.199399,0.000000e+00,2.752895e-03,0.000000e+00
50%,6.942950e-01,0.000000e+00,3.132968e-02,8.143328e-01,5.782230e-03,1.639082e-01,0.000000e+00,9.777229e-03,0.120519,8.851244e-02,...,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.382136,0.000000e+00,5.508129e-03,0.000000e+00
75%,8.648159e-01,8.000000e+00,1.259019e-01,1.002403e+00,8.660590e-03,2.581017e-01,0.000000e+00,1.550507e-01,0.250869,1.843206e-01,...,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.559307,0.000000e+00,8.260448e-03,0.000000e+00
max,1.010000e+00,1.830000e+02,1.324060e+00,1.010000e+00,3.256284e+00,5.482888e+00,8.988807e+00,1.625262e+00,4.191119,1.011162e+01,...,7.000000e+00,1.000000e+00,6.000000e+00,1.000000e+00,1.000000e+00,1.339910e+00,2.229368,1.000000e+00,1.343331e+00,5.300000e+01


#EDA
(피처가 너무 많아서 하나하나 distribution을 보기 어려울 때는 어떻게 해야 할까?
일단 하는데까지 해보고 다른 사람들이 어떻게 했는지 봐야겠다...)

## Train dataset label distribution
0이 1보다 훨씬 많으므로 이를 고려한 평가지표를 사용해야 한다. 

In [ ]:
train_labels_df['target'].value_counts()

0    340085
1    118828
Name: target, dtype: int64

#String columns
첫번째 칼럼은 'customer ID'로, 마지막 16글자만 따와 int64 타입으로 바꾼다.  
두번째 칼렴 'S_2'는 년/월/일로, datetime object로 바꾼다.  
  
'S_2'의 분포를 살펴보면 2017년 3월 - 2018년 3월 총 369일 값을 모두 가진다.  
(아마 customer마다 결제일이 달라서?)  
customer_ID별 'S_2'의 분포를 살펴보면 customer의 85%는 열세달 모두 데이터가 있지만  
나머지 15%는 데이터가 없다. (이 부분을 모델 학습 시킬 때 어떻게 처리하면 될지...)  
날짜는 필요없으므로 (to_period('M') 사용 )년/월만 남겨도 될 것 같다. 또한 년/월대신 1-13까지 숫자를 붙여도 될 것 같다. 




In [ ]:
# String타입 칼럼을 확인
train_str_df = train_df.select_dtypes(include='object')
train_str_df.head()
del train_str_df

# train_df.select_dtypes(include='object').head() 면 한줄이면 됨

,customer_ID,S_2
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16


In [ ]:
# customer_ID의 데이터 타입 변환 (str > int64)
print('변환 전 customer_ID 종류 수: ', train_df['customer_ID'].nunique())
train_df['customer_ID'] = train_df['customer_ID'].str[-16:].apply(int, base=16).astype(np.int64)
print('변환 후 customer_ID 종류 수: ', train_df['customer_ID'].nunique())

In [ ]:
# S_2의 데이터 타입 변환 (str > datetime)
train_df['S_2'] = pd.to_datetime(train_df['S_2'] )
print(train_df['S_2'].dtype)

In [ ]:
# s_2의 분포를 확인 >> 365 + 31 = 396, 13개월 기간의 모든 날짜가 존재. 
print(train_df['S_2'].nunique()) 

396

In [ ]:
# customer_ID 별로 S_2의 분포 확인
customer_data_count = train_df.groupby('customer_ID')['S_2'].count()
customer_data_count.value_counts()
del customer_data_count

13    386034
12     10623
10      6721
9       6411
8       6110
2       6098
11      5961
3       5778
6       5515
7       5198
1       5120
4       4673
5       4671
Name: S_2, dtype: int64

# Integer columns
칼럼별로 unique한 값의 수를 보면 모두 사실 category형 데이터임을 알 수 있다.  
특히 많은 수는 binary이다.  
encoding을 해야할 것 같다.  
  
어떤 피처는 target 피처와 연관이 많을 수 있을 것 같다.  
또한 어떤 피처들은 서로 관련이 있을 수 있을 것 같다. 
correlation으로 확인...  
보통 그 외에 더하기, 곱하기 등으로 관련되는 피처는 어떻게 찾는지? 

In [ ]:
train_int_df = train_df.select_dtypes(include='integer')
print('integer 칼럼의 수:', len(train_int_df.columns))
for col in train_int_df.columns[1:]:
  print('{0} 칼럼 unique한 값의 수: '.format(col)  , train_int_df[col].nunique())

del train_int_df

integer 칼럼의 수: 96
D_39 칼럼 unique한 값의 수:  180
D_44 칼럼 unique한 값의 수:  38
B_4 칼럼 unique한 값의 수:  293
R_2 칼럼 unique한 값의 수:  2
D_49 칼럼 unique한 값의 수:  321
D_51 칼럼 unique한 값의 수:  9
R_3 칼럼 unique한 값의 수:  91
S_6 칼럼 unique한 값의 수:  2
R_4 칼럼 unique한 값의 수:  2
S_8 칼럼 unique한 값의 수:  32
R_5 칼럼 unique한 값의 수:  47
D_59 칼럼 unique한 값의 수:  104
S_11 칼럼 unique한 값의 수:  83
D_63 칼럼 unique한 값의 수:  6
D_64 칼럼 unique한 값의 수:  5
D_65 칼럼 unique한 값의 수:  775
B_16 칼럼 unique한 값의 수:  14
B_19 칼럼 unique한 값의 수:  47
D_66 칼럼 unique한 값의 수:  3
B_20 칼럼 unique한 값의 수:  19
D_68 칼럼 unique한 값의 수:  8
S_13 칼럼 unique한 값의 수:  13
B_22 칼럼 unique한 값의 수:  9
D_70 칼럼 unique한 값의 수:  29
D_72 칼럼 unique한 값의 수:  11
S_15 칼럼 unique한 값의 수:  57
D_74 칼럼 unique한 값의 수:  53
D_75 칼럼 unique한 값의 수:  57
D_78 칼럼 unique한 값의 수:  25
D_79 칼럼 unique한 값의 수:  25
R_8 칼럼 unique한 값의 수:  33
R_9 칼럼 unique한 값의 수:  11
D_80 칼럼 unique한 값의 수:  38
R_10 칼럼 unique한 값의 수:  18
R_11 칼럼 unique한 값의 수:  19
D_81 칼럼 unique한 값의 수:  10
D_82 칼럼 unique한 값의 수:  8
R_13 칼럼 unique한 값의 수:  28
D_83 칼럼 

In [ ]:
train_float_df = train_df.select_dtypes(include='float32')
print('float 칼럼의 수:', len(train_float_df.columns))
for col in train_float_df.columns[1:]:
  print('{0} 칼럼 unique한 값의 수: '.format(col)  , train_float_df[col].nunique())

del train_float_df

float 칼럼의 수: 93
B_1 칼럼 unique한 값의 수:  5319031
B_2 칼럼 unique한 값의 수:  2346669
R_1 칼럼 unique한 값의 수:  4788285
S_3 칼럼 unique한 값의 수:  4004312
D_41 칼럼 unique한 값의 수:  710177
B_3 칼럼 unique한 값의 수:  5273509
D_42 칼럼 unique한 값의 수:  785339
D_43 칼럼 unique한 값의 수:  3708893
D_45 칼럼 unique한 값의 수:  5210401
B_5 칼럼 unique한 값의 수:  5268298
D_46 칼럼 unique한 값의 수:  3388402
D_47 칼럼 unique한 값의 수:  4986282
D_48 칼럼 unique한 값의 수:  4534865
B_6 칼럼 unique한 값의 수:  5152590
B_7 칼럼 unique한 값의 수:  5259879
B_8 칼럼 unique한 값의 수:  88734
D_50 칼럼 unique한 값의 수:  2306378
B_9 칼럼 unique한 값의 수:  5181777
D_52 칼럼 unique한 값의 수:  4949947
P_3 칼럼 unique한 값의 수:  3816101
B_10 칼럼 unique한 값의 수:  4190332
D_53 칼럼 unique한 값의 수:  1428989
S_5 칼럼 unique한 값의 수:  5290271
B_11 칼럼 unique한 값의 수:  5309099
D_54 칼럼 unique한 값의 수:  112645
S_7 칼럼 unique한 값의 수:  4185390
B_12 칼럼 unique한 값의 수:  5193313
D_55 칼럼 unique한 값의 수:  5082515
D_56 칼럼 unique한 값의 수:  2446769
B_13 칼럼 unique한 값의 수:  5249059
D_58 칼럼 unique한 값의 수:  5256972
S_9 칼럼 unique한 값의 수:  2531874
B_14 칼럼 uni

# Feature Engineering
customer_ID 별 데이터로 변환하는 방법.... 